<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=2a279d35a7b1cddd1f87bda4e55c146aa092257933e46fcd9084e74234ce04ab
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-29 12:18:59
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  4.38 K
Current:  1.36 C
-------------------
Today PnL: -5.16 K (-0.04%)
Current PnL: -23.41 L (-15.74%)
CY Booked + Current PnL: -11.97 L (-8.05%)
-------------------
Total profit:  91.62 K
Total loss:  -24.32 L
-------------------
Total Booked + Current PnL: 15.01 L (12.34%)
Total Booked PnL: 38.42 L (31.57%)
Curr Year Booked PnL: 11.44 L (8.4%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.25 C
Est FTT PnL: 89.27 L (65.55%)
Deployed:  1.22 C
Current:  1.36 C
CAGR/XIRR %: 6.78%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
77,TTKPRESTIG,770.00,93.17,35.0,M-SC,2.44,83185.0,-17592.0,17685.0,0.01,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.32,46.0,H-MC,3.62,188690.0,24020.0,19303.0,-0.17,14.59,10.23,26.31,89.0,1.24,1.39,23.73,X40N,NTT,AC
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107855.0,7351.0,21754.0,1.46,7.31,20.17,28.96,79.0,0.34,0.79,43.77,MH,ATH,METALS
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198045.0,-2093.0,22973.0,-0.02,-1.05,11.60,10.44,4.0,-0.09,1.45,4.46,X40,NTT,FMCG
57,RELIANCE,1533.00,-14.04,45.0,H-LC,3.94,215296.0,4930.0,23855.0,0.18,2.34,11.08,13.68,37.0,0.21,1.58,19.25,XY25,NTT,REFINERIES


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-178.41,71.0,M-SC,6.62,90060.0,-12150.0,105541.0,8.91,-11.89,117.19,91.37,208.0,-0.12,0.66,46.20,XY25,NTT,FINANCE
78,UJJIVANSFB,60.00,94.87,54.0,H-SC,10.39,123390.0,-19089.0,38609.0,3.44,-13.40,31.29,13.70,163.0,-0.49,0.91,45.63,OX40N,NTT,BANKS
32,HINDZINC,730.22,23.40,58.0,M-LC,2.94,201100.0,-3976.0,116537.0,2.94,-1.94,57.95,54.89,52.0,-0.03,1.48,21.69,X5K,ATH,METALS
10,BANDHANBNK,400.00,-14.68,42.0,H-SC,4.13,213234.0,-65325.0,322772.0,2.65,-23.45,151.37,92.42,151.0,-0.20,1.57,23.23,XY24,NTT,BANKS
11,BANKINDIA,190.00,-29.19,55.0,H-MC,8.44,182438.0,2630.0,109791.0,1.97,1.46,60.18,62.52,88.0,0.02,1.34,36.05,XR,NTT,BANKS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
71,TANLA,1963.11,-24.44,44.0,H-SC,17.24,202832.0,-67929.0,397875.0,-4.39,-25.09,196.16,121.86,133.0,-0.17,1.49,58.67,AR,ATH,IT
26,GREENPANEL,537.00,193.93,39.0,M-SC,0.99,131248.0,-51830.0,130815.0,-2.55,-28.31,99.67,43.14,230.0,-0.40,0.96,23.46,XY24,NTT,MISC
15,BSOFT,836.99,-19.83,31.0,H-SC,13.82,89964.0,-50689.0,123467.0,-2.31,-36.04,137.24,51.74,131.0,-0.41,0.66,2.59,XR,ATH,IT
38,INDIGOPNTS,1408.00,101.45,29.0,M-SC,1.52,132221.0,-42338.0,42377.0,-2.23,-24.25,32.05,0.02,221.0,-1.00,0.97,15.01,OX40N,NTT,PAINTS
13,BERGEPAINT,680.00,-23.15,33.0,H-MC,4.22,206530.0,-20809.0,68878.0,-2.21,-9.15,33.35,21.14,106.0,-0.30,1.52,16.41,XY24,NTT,PAINTS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WHIRLPOOL,2270.00,-46.09,28.0,M-SC,1.35,91200.0,-298.0,79052.0,-1.35,-0.33,86.68,86.07,223.0,-0.00,0.67,32.44,XR,NTT,DURABLES
37,INDIAMART,4850.92,-55.39,24.0,H-SC,9.98,124476.0,1140.0,132629.0,-0.77,0.92,106.55,108.45,119.0,0.01,0.91,23.48,AR,ATH,MISC


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,94.87,54.0,H-SC,10.39,123390.0,-19089.0,38609.0,3.44,-13.40,31.29,13.70,163.0,-0.49,0.91,45.63,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,35.0,M-SC,2.44,83185.0,-17592.0,17685.0,0.01,-17.46,21.26,0.09,245.0,-0.99,0.61,9.29,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,50.0,H-SC,1.53,220707.0,-48960.0,85303.0,-0.55,-18.16,38.65,13.47,138.0,-0.57,1.62,11.63,XY24,NTT,PAINTS
18,CERA,9475.0,-22.67,36.0,H-SC,1.70,140921.0,-34982.0,76999.0,-0.85,-19.89,54.64,23.89,149.0,-0.45,1.04,21.70,OX40N,NTT,CERAMICS
66,SIS,528.0,2080.83,40.0,H-SC,4.02,88000.0,-23032.0,46640.0,-0.03,-20.74,53.00,21.26,156.0,-0.49,0.65,18.90,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,44.35,55.0,M-MC,10.72,228165.0,3203.0,164461.0,1.13,1.42,72.08,74.53,192.0,0.02,1.68,34.14,XY24,BTT,STEEL
14,BLUESTARCO,2080.00,9.32,46.0,H-MC,3.62,188690.0,24020.0,19303.0,-0.17,14.59,10.23,26.31,89.0,1.24,1.39,23.73,X40N,NTT,AC
84,VOLTAS,1530.00,-2.76,45.0,H-MC,2.86,204090.0,12348.0,25409.0,1.52,6.44,12.45,19.69,99.0,0.49,1.50,13.68,XY25,NTT,AC
17,CAMS,5211.76,-6.27,43.0,H-SC,1.80,106574.0,4570.0,39358.0,0.16,4.48,36.93,43.06,122.0,0.12,0.78,22.36,X40N,ATH,MISC
79,UNIONBANK,163.00,-14.73,51.0,M-LC,6.99,153722.0,12882.0,31605.0,0.84,9.15,20.56,31.59,66.0,0.41,1.13,38.06,XY24,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107855.0,7351.0,21754.0,1.46,7.31,20.17,28.96,79.0,0.34,0.79,43.77,MH,ATH,METALS
11,BANKINDIA,190.00,-29.19,55.0,H-MC,8.44,182438.0,2630.0,109791.0,1.97,1.46,60.18,62.52,88.0,0.02,1.34,36.05,XR,NTT,BANKS
37,INDIAMART,4850.92,-55.39,24.0,H-SC,9.98,124476.0,1140.0,132629.0,-0.77,0.92,106.55,108.45,119.0,0.01,0.91,23.48,AR,ATH,MISC
85,WHIRLPOOL,2270.00,-46.09,28.0,M-SC,1.35,91200.0,-298.0,79052.0,-1.35,-0.33,86.68,86.07,223.0,-0.00,0.67,32.44,XR,NTT,DURABLES
86,WIPRO,420.00,-15.61,37.0,M-LC,5.47,147938.0,-3007.0,112462.0,1.21,-1.99,76.02,72.51,53.0,-0.03,1.09,3.91,XR,NTT,IT


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.52,206228.0,-45540.0,98742.0,0.03,-18.09,47.88,21.13,27.0,-0.46,1.51,10.93,X40,ATH,PAINTS
74,TCS,4476.75,-31.58,29.0,H-LC,12.17,272769.0,-73187.0,148032.0,0.09,-21.15,54.27,21.64,1.0,-0.49,2.00,0.09,X40,ATH,IT
44,JIOFIN,387.00,-5.85,31.0,H-LC,12.80,255936.0,-11429.0,76883.0,0.59,-4.27,30.04,24.48,48.0,-0.15,1.88,48.35,XY24,BTT,FINANCE
83,VBL,671.64,-20.64,32.0,H-LC,8.51,285664.0,-30178.0,142832.0,0.67,-9.55,50.00,35.67,5.0,-0.21,2.10,2.98,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,218057.0,-23867.0,87136.0,-0.46,-9.87,39.96,26.15,8.0,-0.27,1.60,3.04,X40,ATH,IT


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
48,LTIM,7230.2,-7.23,42.0,H-LC,1.58,229972.0,-16179.0,95392.0,0.70,-6.57,41.48,32.18,16.0,-0.17,1.69,27.10,X200,ATH,IT
42,ITC,452.0,-38.77,45.0,H-LC,1.92,198045.0,-2093.0,22973.0,-0.02,-1.05,11.60,10.44,4.0,-0.09,1.45,4.46,X40,NTT,FMCG
65,SIEMENS,4671.5,-3.47,41.0,H-LC,2.41,154975.0,-31120.0,78603.0,0.10,-16.72,50.72,25.51,15.0,-0.40,1.14,14.20,AR,ATH,ELECTRICAL
57,RELIANCE,1533.0,-14.04,45.0,H-LC,3.94,215296.0,4930.0,23855.0,0.18,2.34,11.08,13.68,37.0,0.21,1.58,19.25,XY25,NTT,REFINERIES
19,CIPLA,1795.0,-22.52,39.0,H-LC,4.29,206363.0,1863.0,39560.0,0.46,0.91,19.17,20.25,10.0,0.05,1.52,9.72,X40N,BTT,PHARMA


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.17,272769.0,-73187.0,148032.0,0.09,-21.15,54.27,21.64,1.0,-0.49,2.00,0.09,X40,ATH,IT
80,UNITDSPR,1644.00,-14.66,43.0,H-LC,7.21,223789.0,-10355.0,58968.0,0.29,-4.42,26.35,20.77,86.0,-0.18,1.64,2.19,X40N,NTT,BREWERIES
83,VBL,671.64,-20.64,32.0,H-LC,8.51,285664.0,-30178.0,142832.0,0.67,-9.55,50.00,35.67,5.0,-0.21,2.10,2.98,X40N,ATH,FMCG
30,HCLTECH,1943.91,-4.52,33.0,H-LC,10.03,218057.0,-23867.0,87136.0,-0.46,-9.87,39.96,26.15,8.0,-0.27,1.60,3.04,X40,ATH,IT
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198045.0,-2093.0,22973.0,-0.02,-1.05,11.60,10.44,4.0,-0.09,1.45,4.46,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.47,41.0,H-LC,2.41,154975.0,-31120.0,78603.0,0.10,-16.72,50.72,25.51,15.0,-0.40,1.14,14.20,AR,ATH,ELECTRICAL
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198045.0,-2093.0,22973.0,-0.02,-1.05,11.60,10.44,4.0,-0.09,1.45,4.46,X40,NTT,FMCG
5,ASIANPAINT,3465.66,-18.29,28.0,H-LC,6.52,206228.0,-45540.0,98742.0,0.03,-18.09,47.88,21.13,27.0,-0.46,1.51,10.93,X40,ATH,PAINTS
19,CIPLA,1795.00,-22.52,39.0,H-LC,4.29,206363.0,1863.0,39560.0,0.46,0.91,19.17,20.25,10.0,0.05,1.52,9.72,X40N,BTT,PHARMA
57,RELIANCE,1533.00,-14.04,45.0,H-LC,3.94,215296.0,4930.0,23855.0,0.18,2.34,11.08,13.68,37.0,0.21,1.58,19.25,XY25,NTT,REFINERIES


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4476.75,-31.58,29.0,H-LC,12.17,272769.0,-73187.0,148032.0,0.09,-21.15,54.27,21.64,1.0,-0.49,2.00,0.09,X40,ATH,IT
40,INFY,2275.00,-20.67,40.0,H-LC,8.19,308871.0,-4017.0,175717.0,0.08,-1.28,56.89,54.87,3.0,-0.02,2.27,5.28,X40,BTT,IT
42,ITC,452.00,-38.77,45.0,H-LC,1.92,198045.0,-2093.0,22973.0,-0.02,-1.05,11.60,10.44,4.0,-0.09,1.45,4.46,X40,NTT,FMCG
83,VBL,671.64,-20.64,32.0,H-LC,8.51,285664.0,-30178.0,142832.0,0.67,-9.55,50.00,35.67,5.0,-0.21,2.10,2.98,X40N,ATH,FMCG
1,ABB,7934.00,-40.16,46.0,H-LC,7.68,249955.0,-11664.0,130876.0,0.66,-4.46,52.36,45.57,7.0,-0.09,1.84,6.36,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-47.69,43.0,L-SC,40.28,78912.0,-34637.0,74690.0,1.66,-30.50,94.65,35.27,268.0,-0.46,0.58,93.14,XR,NTT,HOTELS
49,MASFIN,398.61,-20.96,35.0,H-SC,12.28,89910.0,-8070.0,29670.0,-0.97,-8.24,33.00,22.05,152.0,-0.27,0.66,30.69,XR,ATH,FINANCE
61,SAMMAANCAP,326.00,-178.41,71.0,M-SC,6.62,90060.0,-12150.0,105541.0,8.91,-11.89,117.19,91.37,208.0,-0.12,0.66,46.20,XY25,NTT,FINANCE
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107855.0,7351.0,21754.0,1.46,7.31,20.17,28.96,79.0,0.34,0.79,43.77,MH,ATH,METALS
78,UJJIVANSFB,60.00,94.87,54.0,H-SC,10.39,123390.0,-19089.0,38609.0,3.44,-13.40,31.29,13.70,163.0,-0.49,0.91,45.63,OX40N,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SHALBY,327.00,1164.75,63.0,M-SC,13.90,172086.0,-9057.0,54517.0,0.48,-5.00,31.68,25.10,235.0,-0.17,1.26,37.91,XY24,NTT,HEALTHCARE
61,SAMMAANCAP,326.00,-178.41,71.0,M-SC,6.62,90060.0,-12150.0,105541.0,8.91,-11.89,117.19,91.37,208.0,-0.12,0.66,46.20,XY25,NTT,FINANCE
60,SAIL,228.00,44.35,55.0,M-MC,10.72,228165.0,3203.0,164461.0,1.13,1.42,72.08,74.53,192.0,0.02,1.68,34.14,XY24,BTT,STEEL
81,VAIBHAVGBL,521.00,49.86,46.0,H-SC,5.78,129714.0,-53061.0,164127.0,-1.07,-29.03,126.53,60.77,125.0,-0.32,0.95,16.18,XR,NTT,JEWELLERY
50,NATIONALUM,244.55,-46.15,51.0,H-MC,11.85,107855.0,7351.0,21754.0,1.46,7.31,20.17,28.96,79.0,0.34,0.79,43.77,MH,ATH,METALS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.31
1,25,43.76
2,50,75.03


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.67
LC    32.60
MC    23.74
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.31
X40      14.80
X40N     11.42
XR       11.12
XY25     10.26
AR        8.23
OX40N     7.90
X200      1.69
MH        1.60
X5K       1.48
SR        1.20
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    29.21
H-LC    26.05
H-MC    20.78
M-SC    13.00
M-LC     5.49
M-MC     2.66
L-SC     1.46
L-LC     1.06
L-MC     0.30
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,14.14,-6.60,41.78
IT,12.75,-21.12,87.16
FINANCE,10.09,-19.41,71.51
MISC,6.98,-20.87,88.07
BANKS,6.23,-17.70,79.62
PAINTS,5.62,-20.59,38.57
ELECTRICAL,5.49,-10.29,48.99
INSURANCE,3.77,-7.85,48.17
AC,3.60,1.07,15.49


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3256749.0,22
XR,1395938.0,14
AR,1210756.0,9
X40,833249.0,10
X40N,650441.0,9
OX40N,572590.0,10
XY25,442515.0,7
SR,257921.0,2
X5K,116537.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3815685.0,29
M-SC,1476238.0,17
H-MC,1279105.0,15
H-LC,1272799.0,15
M-LC,403137.0,4
M-MC,316166.0,2
L-SC,262839.0,3
L-MC,61187.0,1
L-LC,40047.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1257070.0      6
M-SC       XY24       848915.0      7
H-SC       AR         829562.0      5
           XR         811605.0      7
H-MC       XY24       583577.0      4
H-LC       X40        532600.0      5
H-SC       X40N       321544.0      4
M-MC       XY24       316166.0      2
H-SC       OX40N      264622.0      4
           SR         257921.0      2
H-LC       X40N       241360.0      3
H-MC       X40        222973.0      4
H-LC       AR         209479.0      2
H-MC       XY25       179842.0      2
M-LC       XY24       174138.0      2
L-SC       XR         173500.0      2
M-SC       AR         171715.0      2
           OX40N      144998.0      4
           XR         127393.0      2
H-LC       XY25       117085.0      3
M-LC       X5K        116537.0      1
           XR         112462.0      1
H-MC       XR         109791.0      1
M-SC       XY25       105541.0      1
H-LC       X200        95392.0      1
L-SC       OX40N       89339.0      1
H-MC       X40N        87537.0      2
M-SC       X40         77676.0      1
H-LC       XY24        76883.0      1
H-MC       OX40N       73631.0      1
H-SC       MH          73361.0      1
L-MC       XR          61187.0      1
L-LC       XY25        40047.0      1
H-MC       MH          21754.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 25.0 seconds
